# Conclusions

I think a tool like this could be a very useful addition at some point, especially for profiling plugin performance. It would be possible to extend this to provide rough profiling (for instance be allowing for 'notes' to be attached to traces) but that might be overkill. It's more useful to think of this as a diagnostics tool, and when particular slowness is encountered heavier profiling tools can be brought to bear.

## Pain points

Threading trace IDs through the call graph is a pain. Much of this is dependent on how multi-threaded we are, and on various unnecessary convolutions in our graph itself.

## Alternative designs

The current design is fairly simple: each process checks incoming messages for the existence of a `trace_id` field, and if it exists that process begins sending timestamps as well. When it receives a message with no `trace_id` field, it stops sending them.

In the client, we begin tracing by just adding that field to our messages.

When we want to _collect_ traces, we have a special RPC `xi-rpc.collect_traces`, which we send out to all peers; they respond with whatever traces they've collected. This is annoying, becuase we have to propogate this call up through the handler to each plugin process.

Ideally we would have some mechanism by which peers automatically returned traces periodically, and then we would stash them in some global repository.

### Always on / autosend?
At least at the RPC level, it should be possible to implement very efficient tracing, taking advantage of the idle scheduler.

- _all_ RPCs would get a timestamp
- each runloop would maintain a thread-local array of traces
- if ever the idle checker fired and found no tasks, the process would send its accumulated traces to its peer (this assumes some mechanism whereby a process knows whether or not it is a client or not)
- core aggregates traces, discarding them if they aren't needed.

This could be done without a lot of overhead. 


In [1]:
import os
import json
from collections import namedtuple, Counter, defaultdict
from operator import attrgetter

In [2]:
class Trace(object):
    def __init__(self, label, parent, proc_name, timestamp):
        self.label = label
        self.parent_time = parent
        self.proc_name = proc_name
        self.timestamp = timestamp
        self.children = []
        self.ident = -1
        self.delta_t = 0
        self.num_ancestors = -1
        self.parent = None
        

    def __str__(self):
        return "{:12} -> {:<12} (+{:>10.2f}µs) {}.{}".format(self.parent_time, self.timestamp, self.delta_t / 1000, self.proc_name, self.label)
        
    def __repr__(self):
        return "{}:".format(self.ident) + str(self)
    
    def group_print(self, level=0):
        fstr = "{}{:<%s}{}" % (12 - level,)
        print(fstr.format(' ' * level, str(self.ident) + ':', str(self)))
        level += 1
        for child in self.children:
            child.group_print(level)

In [3]:
trace_file = './trace_2.json'

In [4]:
traces = json.load(open(trace_file))
traces = [Trace(t['label'], t['parent'], t['proc_name'], t['timestamp']) for t in traces]

min_t = min(t.timestamp for t in traces)
traces.sort(key=attrgetter('timestamp'))

# the last item is the trace request
traces.pop()

for (i, t) in enumerate(traces):
    t.ident = i + 1
    t.timestamp = max(0, t.timestamp - min_t + 1)
    t.parent_time = max(0, t.parent_time - min_t + 1)
    if i > 0:
        t.delta_t = t.timestamp - traces[i-1].timestamp

In [5]:
len_t = len(traces)

print("loaded {} traces from {}".format(len_t, set(t.proc_name for t in traces)))

set_t = set(t.timestamp for t in traces)
if len(set_t) != len_t:
    print("found duplicate traces")
    print(Counter(traces).most_common())
for t in traces[:10]:
    print(t)


loaded 3865 traces from {'xi-mac', 'xi-core'}
           0 -> 1            (+      0.00µs) xi-mac.send.scroll_page_down
           1 -> 115397       (+    115.40µs) xi-core.recv.edit
      115397 -> 326732       (+    211.34µs) xi-core.send.update
      115397 -> 387659       (+     60.93µs) xi-core.send.scroll_to
      326732 -> 530768       (+    143.11µs) xi-mac.recv.update
      387659 -> 553944       (+     23.18µs) xi-mac.recv.scroll_to
           0 -> 1968143774   (+1967589.83µs) xi-mac.send.click
  1968143774 -> 1968283435   (+    139.66µs) xi-core.recv.edit
  1968283435 -> 1968505496   (+    222.06µs) xi-core.send.update
  1968283435 -> 1968567792   (+     62.30µs) xi-core.send.scroll_to


## Making some sense of data

What can we do with these?

1. Causality trees: each event has a maximum of 1 parent.
2. classify like event chains: how many distinct patterns of messages are there?
3. classify event pairs; show their average times


In [6]:
def call_trees(traces):
    child_counts = Counter([t.parent_time for t in traces])
    grouped_traces = {t.timestamp: t for t in traces}

    while True:
        childless = [t for (i, t) in grouped_traces.items() if t.parent_time != 0 and child_counts.get(i, 0) <= 0]
        orphans = [t for t in childless if t.parent_time not in grouped_traces]
        assert not len(orphans), '\n'.join(str(o) for o in orphans)
        if len(childless) == 0:
            break
        for trace in childless:
            del grouped_traces[trace.timestamp]
            grouped_traces[trace.parent_time].children.append(trace)
            trace.parent = grouped_traces[trace.parent_time]
            trace.delta_t = trace.timestamp - trace.parent.timestamp
            child_counts[trace.parent_time] -= 1

    grouped = list(grouped_traces.values())
    for g in grouped:
        g.num_ancestors = num_ancestors(g)
    return grouped

def num_ancestors(node):
    if not len(node.children):
        return 1
    else:
        return 1 + sum(num_ancestors(c) for c in node.children)

def group_print(call, level=0):
    if level == 0:
        elapsed = max_child_time(call) - call.timestamp
        print('total elapsed: {:.2f}µs'.format(elapsed/1000))

    fstr = "{}{:<%s}{}" % (12 - level,)
    print(fstr.format(' ' * level, str(call.ident) + ':', str(call)))
    level += 1
    for child in call.children:
        child.group_print(level)

        
def max_child_time(node):
    if not len(node.children):
        return node.timestamp
    else:
        return max([max_child_time(c) for c in node.children])

In [7]:
calls = call_trees(traces)

# call trees

These group all traces which share a common ancestor. A single trace can have multiple children; for instance `xi-core.recv.update` leads to `xi-core.send.scroll_to` _and_ `xi-core.send.update` (as well as other calls to plugins)

Time deltas (+ xxx µs) are relative to the parent.

In [8]:
group_print(calls[1])

total elapsed: 601.52µs
7:                     0 -> 1968143774   (+1967589.83µs) xi-mac.send.click
 8:           1968143774 -> 1968283435   (+    139.66µs) xi-core.recv.edit
  9:          1968283435 -> 1968505496   (+    222.06µs) xi-core.send.update
   11:        1968505496 -> 1968719547   (+    214.05µs) xi-mac.recv.update
  10:         1968283435 -> 1968567792   (+    284.36µs) xi-core.send.scroll_to
   12:        1968567792 -> 1968745297   (+    177.50µs) xi-mac.recv.scroll_to


In [9]:
group_print(calls[-1])

total elapsed: 82.21µs
3863:                  0 -> 39468675823  (+   1442.31µs) xi-mac.send.request_lines
 3864:       39468675823 -> 39468719147  (+     43.32µs) xi-core.recv.edit
  3865:      39468719147 -> 39468758036  (+     38.89µs) xi-core.send.update


In [10]:
group_print(calls[-16])

total elapsed: 98.49µs
3809:                  0 -> 39258794286  (+  19447.78µs) xi-mac.send.scroll
 3810:       39258794286 -> 39258892773  (+     98.49µs) xi-core.recv.edit


## Event pairs (send/recv)

Let's start with pairs of items: for every node, if it has a parent, add (parent, child) to some index of identifiers.

In [11]:
pairs = defaultdict(list)
for node in traces:
    if node.parent is not None:
        ident = "{}.{} -> {}.{}".format(node.parent.proc_name,
                                      node.parent.label,
                                      node.proc_name,
                                      node.label)
        pairs[ident].append(node)


## average inter-event time

For common event pairs (recv/send, send/recv) what is the average time between events for occurances of that pair?

In [12]:
results = []
for (k, v) in pairs.items():
    if len(v) < 5:
        continue
    times = [p.timestamp - p.parent.timestamp for p in v]
    results.append((k, times))

avgs = [(k, sum(v)/len(v)) for k, v in results]
avgs.sort(key=lambda kv: kv[1])

for i, (k, avg) in enumerate(avgs):
    print("({:<4} samples){:>10.2f} µs: {}".format(len(results[i][1]), avg/1000, k, ))

(168  samples)     77.29 µs: xi-mac.send.request_lines -> xi-core.recv.edit
(528  samples)     87.15 µs: xi-mac.send.scroll_page_down -> xi-core.recv.edit
(169  samples)    100.23 µs: xi-mac.send.scroll -> xi-core.recv.edit
(527  samples)    187.44 µs: xi-core.send.update -> xi-mac.recv.update
(169  samples)    196.35 µs: xi-core.send.scroll_to -> xi-mac.recv.scroll_to
(894  samples)    203.27 µs: xi-core.recv.edit -> xi-core.send.update
(173  samples)    361.01 µs: xi-core.recv.edit -> xi-core.send.scroll_to


## As a graph:

In [13]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
from bokeh.models import PrintfTickFormatter
from bokeh.models import HoverTool
import colorcet as cc
import math
import random

output_notebook()

def label_for_event(event):
    par = "{}.{}".format(event.parent.proc_name, event.parent.label)
    child = "{}.{}".format(event.proc_name, event.label)
#     if len(par) > 16:
#         par = par[:7] + '..' + par[-7:]
#     if len(child) > 16:
#         child = child[:7] + '..' + child[-7:]
    return "{}->{}".format(par, child)



to_graph = [(label_for_event(v[0]), v) for k ,v in pairs.items() if len(v) > 5]
to_graph.sort(key=lambda kv: kv[0])
keys = [k for k,_ in to_graph]

interval = int(len(cc.rainbow) / len(to_graph))
palette = [cc.rainbow[i*interval] for i in range(len(to_graph) + 1)]

p = figure(y_range=keys, plot_width=900, plot_height=600, title="RPC time-to-receive, by route",
          toolbar_location='right')
p.xaxis[0].formatter = PrintfTickFormatter(format="%dµs")

for i, (k, v) in enumerate(to_graph):
    times = [(p.timestamp - p.parent.timestamp)/1000 for p in v]
    color = palette[i]
    cr = p.circle(times, [k]*len(times), color=color, fill_alpha=0.5, hover_alpha=1.0)

show(p)


Loading BokehJS ...

![graph_1.png](./graph_2.png)

That doesn't tell us _too_ much. Over this small trace, we can glean a bit of interesting stuff:

- The syntect plugin is frequently responding quickly; 
- the longest average operation is between receiving an update in core and updating plugins
- xi-mac takes significantly longer to process `scroll_to` than to process `update`
